# **Text generation in the style of The Metamorphosis** #

## **Import necessary dependencies** ##

In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


## **Load and preprocess the data** ##

In [2]:
# load the text file and convert them all into lower case
filename = "data/kafka.txt"
raw_text = open(filename, encoding='utf-8').read()
raw_text = raw_text.lower()

# print out the first 1000 characters
raw_text[: 1000]

'\ufeffthe project gutenberg ebook of metamorphosis, by franz kafka\ntranslated by david wyllie.\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.net\n\n** this is a copyrighted project gutenberg ebook, details below **\n**     please follow the copyright guidelines in this file.     **\n\n\ntitle: metamorphosis\n\nauthor: franz kafka\n\ntranslator: david wyllie\n\nrelease date: august 16, 2005 [ebook #5200]\nfirst posted: may 13, 2002\nlast updated: may 20, 2012\n\nlanguage: english\n\n\n*** start of this project gutenberg ebook metamorphosis ***\n\n\n\n\ncopyright (c) 2002 david wyllie.\n\n\n\n\n\n  metamorphosis\n  franz kafka\n\ntranslated by david wyllie\n\n\n\ni\n\n\none morning, when gregor samsa woke from troubled dreams, he found\nhimself transformed in his bed into a horrible v

In [3]:
# get the list of all unique characters
chars = sorted(list(set(raw_text)))

# map between characters and their index, in both directions
# so we can encode and decode between them
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# print out all unique characters
print(chars)

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
# number of raw characters and
# number of unique characters in the text
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  139056
Total Vocab:  58


In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
skip = 2
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, skip):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  69478


In [6]:
# reshape X to be [samples, time steps, features] to 
# feed into LSTM network
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize the data
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## **Model training** ##

In [7]:
# build the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[
          2]), return_sequences=True, implementation=2))
model.add(Dropout(0.2))
model.add(GRU(256, implementation=2))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# save the model after each epoch if there is an improvement
# (the loss reduced) for easily retrain and reload
filepath = "model_checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}-bigger-continuing.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# fit the model to the data
# model.fit(X, y, epochs=24, batch_size=64, callbacks=callbacks_list, verbose=0)

## **Using LSTM network to generate text** ##

In [8]:
# load the network weights from the best epoch
filename = "model_checkpoints/weights-improvement-18-1.4017-bigger-continuing.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
# pick a random pattern 
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]

# print out the pattern in human-readable form
print ("The context:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("-------------------------------------------------------------------")
print("\nGenerated text: \n", "\"")


# AI writer in action!
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\n\"", "\nDone.")

The context:
" elf and slammed the door shut
again from outside.  but she seemed to regret her behaviour, as she
op "
-------------------------------------------------------------------

Generated text: 
 "
oeec the rall whrh the saie was she was and say the door of his poom and then he had been giaic oo his room.  the foorr would had eeen aala to fet the foor of the lookhe, but whnh his sister wasted to be help fomm.  he had never tple tha rhiet of his look on his back in the eoor of the booree and the suairs what whey was and santnd the tay the doorr wiat was not oo the counle oo cetee that he cada to coone the door and whth him monk on his room, ald he would have been aalk to miv his hin hn his 
" 
Done.
